In [196]:
##################################################################
# Использование API для получения данных о музейных коллекциях
##################################################################


##################################################################
# Описание API
#
# Ключ: для использования выбранного API ключ не требуется.
#
# Всего есть 4 основных раздела:
#
# Объект (Object) – данные об объекте (например, о картине),
# Департаменты (Departments) –  данные о департаменте/разделе 
# (например, "Современное искусство"),
# Поиск (Search) – используется для поиска,
# В рамках задач поработаем с блоками Object и Search.
#
# С более подробным описанием каждого доступного параметра можно 
# ознакомиться в документации: https://metmuseum.github.io/
##################################################################

In [197]:
# Импортируем библиотеки

import requests
import pandas as pd

In [198]:
# Найдем все произведения искусств, связанные с 'Подсолнухами'

QUERY = 'sunflowers'
r = requests.get('https://collectionapi.metmuseum.org/public/collection/v1/search?q=' + QUERY)
res_sun = r.json()
res_sun

{'total': 85,
 'objectIDs': [436524,
  484935,
  437112,
  210191,
  431264,
  397949,
  656530,
  480725,
  486590,
  375281,
  485308,
  705155,
  11922,
  2032,
  816522,
  343052,
  20141,
  208554,
  403496,
  2019,
  360837,
  207869,
  437115,
  400581,
  423400,
  707887,
  223828,
  682927,
  57922,
  649813,
  79739,
  193938,
  339347,
  208218,
  437984,
  202228,
  436534,
  761604,
  437329,
  436252,
  436580,
  363282,
  190739,
  207753,
  437526,
  347980,
  822570,
  487043,
  203893,
  367434,
  423237,
  839296,
  262423,
  367817,
  13644,
  820022,
  648617,
  761731,
  814626,
  207967,
  626832,
  262430,
  262428,
  704667,
  696949,
  707697,
  205386,
  262378,
  426804,
  664519,
  436121,
  436529,
  633526,
  747779,
  369073,
  436144,
  852414,
  427872,
  436530,
  436535,
  437980,
  724844,
  834585,
  834580,
  834765]}

In [199]:
# Переведем полученный словарь в датасет

df = pd.DataFrame(res_sun)
df.head()

,total,objectIDs
0,85,436524
1,85,484935
2,85,437112
3,85,210191
4,85,431264


In [200]:
# Найдём в датасете полученных произведений требуемые нам по 'id_search'

id_search = [437329, 2032, 20149, 436524, 16822570]
id_search = pd.DataFrame(id_search).rename(columns={0: 'id'})
id_search

,id
0,437329
1,2032
2,20149
3,436524
4,16822570


In [201]:
find_id = df.query('objectIDs in @id_search.id')
find_id

,total,objectIDs
0,85,436524
13,85,2032
38,85,437329


In [202]:
# Соберем информацию об объекте '437980'

In [203]:
url = 'https://collectionapi.metmuseum.org/public/collection/v1/objects/'
obj_id = '437980'
r = requests.get(url+obj_id)
res = r.json()
res

{'objectID': 437980,
 'isHighlight': False,
 'accessionNumber': '49.30',
 'accessionYear': '1949',
 'isPublicDomain': True,
 'primaryImage': 'https://images.metmuseum.org/CRDImages/ep/original/DP130999.jpg',
 'primaryImageSmall': 'https://images.metmuseum.org/CRDImages/ep/web-large/DP130999.jpg',
 'additionalImages': [],
 'constituents': [{'constituentID': 161947,
   'role': 'Artist',
   'name': 'Vincent van Gogh',
   'constituentULAN_URL': 'http://vocab.getty.edu/page/ulan/500115588',
   'constituentWikidata_URL': 'https://www.wikidata.org/wiki/Q5582',
   'gender': ''}],
 'department': 'European Paintings',
 'objectName': 'Painting',
 'title': 'Cypresses',
 'culture': '',
 'period': '',
 'dynasty': '',
 'reign': '',
 'portfolio': '',
 'artistRole': 'Artist',
 'artistPrefix': '',
 'artistDisplayName': 'Vincent van Gogh',
 'artistDisplayBio': 'Dutch, Zundert 1853–1890 Auvers-sur-Oise',
 'artistSuffix': '',
 'artistAlphaSort': 'Gogh, Vincent van',
 'artistNationality': 'Dutch',
 'artistB

In [204]:
# Выведем необходимую нам информацию об объекте '437980'

object_info = [res['objectID'], res['objectName'], res['title'], res['culture'], res['GalleryNumber'], \
               res['artistDisplayName'], res['department']]
object_info

[437980,
 'Painting',
 'Cypresses',
 '',
 '825',
 'Vincent van Gogh',
 'European Paintings']

In [205]:
# Создадим пустой датафрейм 

data = pd.DataFrame()

In [206]:
# Теперь напишем цикл, с помощью которого будет собрана информация об объектах, 
# которые были получены для всех произведений искусств, связанных с 'Подсолнухами'.
# Результаты запишем в датафрейм, а именно – следующие параметры:
#
# objectID – id объекта,
# title – название,
# artistDisplayName – автор,
# department – департамент,
# objectBeginDate – дата (начало),
# objectEndDate – дата (конец),
# period – название периода,
# objectName – название/категория объекта,
# culture – культура

for i in res_sun['objectIDs']:
    request_object = requests.get(f'https://collectionapi.metmuseum.org/public/collection/v1/objects/{i}')
    object_from_request = request_object.json()
    try:
        df =  pd.DataFrame({ 'objectID' : object_from_request['objectID'],
                      'title' : object_from_request['title'],
                      'artistDisplayName' : object_from_request['artistDisplayName'],
                      'department' : object_from_request['department'],
                      'objectBeginDate' : object_from_request['objectBeginDate'],
                      'objectEndDate' : object_from_request['objectEndDate'],
                      'period' : object_from_request['period'],
                      'objectName' : object_from_request['objectName'],
                      'culture' : object_from_request['culture']}, index=[0])
        data = data.append(df)
    except:
        continue

In [207]:
# Найдем сколько объектов относятся к японской культуре ('culture = "Japan" ')

count_japan = data.query('culture == "Japan"')
count_japan

,objectID,title,artistDisplayName,department,objectBeginDate,objectEndDate,period,objectName,culture
0,57922,Incense box,,Asian Art,1700,1799,Edo period (1615–1868),Incense box,Japan


In [208]:
# Выполним поисковый запрос для департамента "Asian art" ('departmentId' = 6) с тегом "cat" 
# и запишем 'id' объектов в 'cat_ids'. Используем полученный ранее цикл, чтобы получить 
# данные о первой 1000 объектов из списка

request_by_search = requests.get('https://collectionapi.metmuseum.org/public/collection/v1/search?departmentId=6&q=cat')
cats_id = request_by_search.json()


asian_art_data = pd.DataFrame()

count = 0

for id_number in cats_id['objectIDs']:
    request_by_search = requests.get(f'https://collectionapi.metmuseum.org/public/collection/v1/objects/{id_number}')
    cats = request_by_search.json()
    try:
        df_asian_art =  pd.DataFrame({ 'objectID' : cats['objectID'],
                      'title' : cats['title'],
                      'artistDisplayName' : cats['artistDisplayName'],
                      'department' : cats['department'],
                      'objectBeginDate' : cats['objectBeginDate'],
                      'objectEndDate' : cats['objectEndDate'],
                      'period' : cats['period'],
                      'objectName' : cats['objectName'],
                      'culture' : cats['culture']}, index=[0])      
        asian_art_data = asian_art_data.append(df_asian_art)
    except:
        continue
    count += 1
    if count > 999:
        break

In [209]:
# Найдем в каком периоде ('period') было выполнено наибольшее количество работ, представленных в музее

max_arts_by_period = asian_art_data.period.value_counts().sort_values(ascending=False).head(1)
max_arts_by_period

Edo period (1615–1868)    455
Name: period, dtype: int64